# CSCE 5290 - Final Project
## Pre-trained model evaluation (BERT, T5)
Dan Waters (danwaters@my.unt.edu)

In [3]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
     |████████████████████████████████| 895 kB 69.4 MB/s 
     |████████████████████████████████| 3.3 MB 26.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [9]:
# Get the data (not the one with the start tokens)
!gdown --id 1zzLvOimTsFtKkm4DjznAQFiSQ4smRgpj

Downloading...
From: https://drive.google.com/uc?id=1zzLvOimTsFtKkm4DjznAQFiSQ4smRgpj
To: /content/cnn_cleaned_50k.csv
100% 113M/113M [00:00<00:00, 170MB/s]


In [10]:
import pandas as pd
cnn_df = pd.read_csv('cnn_cleaned_50k.csv')
cnn_df = cnn_df[['text', 'summary']]
cnn_df.head(5)

,text,summary
0,by associated press published 14:11 est 25 oct...,bishop john folda of north dakota is taking ti...
1,ralph mata was an internal affairs lieutenant...,criminal complaint cop used his role to help c...
2,he been accused of making many fashion faux pa...,prime minister and his family are enjoying an ...
3,beirut syria carried out an airstrike on refug...,airstrike kills nine syrians in refugee camp s...
4,kabul afghanistan china top security official ...,china top security official visited afghanista...


In [11]:
article = cnn_df.iloc[0]['text']
article

'by associated press published 14:11 est 25 october 2013 updated 15:36 est 25 october 2013 the bishop of the fargo catholic diocese in north dakota has exposed potentially hundreds of church members in fargo grand forks and jamestown to the hepatitis virus in late september and early october the state health department has issued an advisory of exposure for anyone who attended five churches and took communion bishop john folda pictured of the fargo catholic diocese in north dakota has exposed potentially hundreds of church members in fargo grand forks and jamestown to the hepatitis state immunization program manager molly howell says the risk is low but officials feel it important to alert people to the possible exposure the diocese announced on monday that bishop john folda is taking time off after being diagnosed with hepatitis the diocese says he contracted the infection through contaminated food while attending conference for newly ordained bishops in italy last month symptoms of h

### Write the results to files to compare with ROUGE. 

Limit of 100 documents to conserve time.

In [17]:
# Summarize a bunch of articles
score_max_count = 100

with open('cnn_pred_bart.txt', 'w') as f_pred:
  for i, c in cnn_df.iterrows():
    text = cnn_df.iloc[i]['text']
    s = summarizer(text, max_length=250, min_length=25, do_sample=False)
    f_pred.write(s[0]['summary_text'])
    if i >= score_max_count:
      break

Your max_length is set to 250, but you input_length is only 226. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 234. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 193. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 194. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 183. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 232. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 243. You might c

In [19]:
with open('cnn_ref.txt', 'w') as f_ref:
  for i, c in cnn_df.iterrows():
    summary = cnn_df.iloc[i]['summary']
    f_ref.write(summary)
    if i >= score_max_count:
      break

In [20]:
!pip install rouge

In [21]:
from rouge import FilesRouge
import sys
fr = FilesRouge()
#sys.setrecursionlimit(50 * 50 + 10)
scores = fr.get_scores('cnn_pred_bart.txt', 'cnn_ref.txt', avg=True)

In [22]:
scores

{'rouge-1': {'f': 0.5367860551290475,
  'p': 0.4675379426644182,
  'r': 0.6301136363636364},
 'rouge-2': {'f': 0.20921725925819867,
  'p': 0.17614452884417983,
  'r': 0.25758030621435},
 'rouge-l': {'f': 0.4728944772006834,
  'p': 0.41188870151770657,
  'r': 0.5551136363636363}}

### Compare with Google's T5 transformer (pretrained on some data including CNN/Dailymail)
see: https://huggingface.co/transformers/task_summary.html

In [23]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)

outputs = model.generate(inputs["input_ids"], 
                         max_length=250, min_length=25, length_penalty=2.0, 
                         num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0]))

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

<pad> the bishop of the fargo catholic diocese in north dakota has exposed potentially hundreds of church members in fargo grand forks and jamestown to the hepatitis virus in late september and early october. the risk is low but officials feel it important to alert people to the possible exposure the diocese announced on monday that bishop john folda is taking time off after being diagnosed with hepatitis.</s>


In [26]:
# Summarize a bunch of articles
from tqdm import tqdm
score_max_count = 100

with open('cnn_pred_t5.txt', 'w') as f_pred:
  for i, c in tqdm(cnn_df.iterrows(), total = cnn_df.shape[0]):
    text = cnn_df.iloc[i]['text']

    inputs = tokenizer("summarize: " + text, return_tensors="pt", 
                       max_length=512, truncation=True)
    
    outputs = model.generate(inputs["input_ids"], 
                         max_length=250, min_length=25, length_penalty=2.0, 
                         num_beams=4, early_stopping=True)
    
    s = tokenizer.decode(outputs[0]).replace('<pad>', '').replace('</s>', '')

    f_pred.write(s)
    if i >= score_max_count:
      break

  0%|          | 100/50000 [28:43<238:50:14, 17.23s/it]


In [28]:
scores = fr.get_scores('cnn_pred_t5.txt', 'cnn_ref.txt', avg=True)
scores

{'rouge-1': {'f': 0.5383877110517572, 'p': 0.4659468438538206, 'r': 0.6375},
 'rouge-2': {'f': 0.21450727270994213,
  'p': 0.1779845575133637,
  'r': 0.2698889222455719},
 'rouge-l': {'f': 0.47744721201145,
  'p': 0.41320598006644516,
  'r': 0.5653409090909091}}